# TML Database → matches.csv & points_sets_games.csv (UI)
Aquest notebook baixa dades de **TML-Database** (GitHub) i construeix els fitxers:
- `data/matches.csv`
- `data/points_sets_games.csv`

Inclou una **mini UI** per triar anys, incloure tornejos en curs i randomitzar l'orientació A/B.

> ℹ️ Llicència: TML-Database es basa en Jeff Sackmann i s'ofereix amb condicions d'ús pròpies. Revisa la llicència del repo si l'ús és comercial.


## 1) Dependències

In [11]:

# Si falta ipywidgets, descomenta:
# !pip install ipywidgets requests pandas numpy --quiet

import os, io, requests, pandas as pd, numpy as np, datetime as dt
from IPython.display import display, HTML
try:
    import ipywidgets as widgets
except Exception as e:
    print("ip ywidgets no disponible. Pots seguir executant amb la cel·la de funcions i crides manualment.")


## 2) Funcions d'ingesta des de TML

In [12]:

RAW_BASE = "https://raw.githubusercontent.com/Tennismylife/TML-Database/master"

def year_range(s: str):
    s = str(s)
    if "-" in s:
        a,b = s.split("-",1)
        return list(range(int(a), int(b)+1))
    return [int(s)]

def fetch_csv(url: str) -> pd.DataFrame:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    # TML files are comma-separated
    return pd.read_csv(io.StringIO(r.text))

def yyyymmdd_to_date(x):
    try:
        x = str(int(x)).strip()
        return f"{x[:4]}-{x[4:6]}-{x[6:8]}"
    except Exception:
        return None
        
# --- PATCH: norm_surface robusta + typo "coce" arreglat a "coerce" ---
import pandas as pd
import numpy as np

def norm_surface(s):
    if pd.isna(s):
        return ""
    try:
        s = str(s).strip().lower()
    except Exception:
        return ""
    if "hard" in s: return "hard"
    if "clay" in s: return "clay"
    if "grass" in s: return "grass"
    if "carpet" in s: return "indoor-hard"
    return ""

def map_level(lvl):
    m = {"G":"GS", "A":"ATP", "D":"Davis", "F":"Finals"}
    return m.get(str(lvl).strip(), str(lvl).strip())

def yyyymmdd_to_date(x):
    try:
        x = str(int(x)).strip()
        return f"{x[:4]}-{x[4:6]}-{x[6:8]}"
    except Exception:
        return None

def build_from_tml(df: pd.DataFrame, seed: int = 2025, do_flip: bool = True):
    cols_req = [
        "tourney_id","tourney_name","surface","tourney_level","tourney_date","match_num",
        "winner_id","winner_ioc","loser_id","loser_ioc","best_of","round","minutes",
        "w_ace","w_df","w_svpt","w_1stIn","w_1stWon","w_2ndWon","w_SvGms","w_bpSaved","w_bpFaced",
        "l_ace","l_df","l_svpt","l_1stIn","l_1stWon","l_2ndWon","l_SvGms","l_bpSaved","l_bpFaced"
    ]
    for c in cols_req:
        if c not in df.columns:
            df[c] = np.nan

    df["date"] = df["tourney_date"].apply(yyyymmdd_to_date)
    df["surface_norm"] = df["surface"].apply(norm_surface)
    df["level_norm"] = df["tourney_level"].apply(map_level)
    df["indoor_flag"] = (df["surface_norm"]=="indoor-hard").astype(int)
    df["best_of_5"] = (pd.to_numeric(df["best_of"], errors="coerce")==5).astype(int)
    df["minutes"] = pd.to_numeric(df["minutes"], errors="coerce")
    df["match_id"] = df.apply(lambda r: f"{r.get('tourney_id','')}_{r.get('match_num','')}_{r.get('date','')}", axis=1)

    rng = np.random.default_rng(seed)
    flip = rng.integers(0,2, size=len(df)).astype(bool) if do_flip else np.zeros(len(df), dtype=bool)

    player_a_id = df["winner_id"].astype(str).where(~flip, df["loser_id"].astype(str))
    player_b_id = df["loser_id"].astype(str).where(~flip, df["winner_id"].astype(str))
    player_a_country = df["winner_ioc"].astype(str).where(~flip, df["loser_ioc"].astype(str))
    player_b_country = df["loser_ioc"].astype(str).where(~flip, df["winner_ioc"].astype(str))

    matches = pd.DataFrame({
        "match_id": df["match_id"],
        "date": df["date"],
        "tournament": df["tourney_name"].astype(str),
        "city": df["tourney_name"].astype(str),
        "country": "",
        "level": df["level_norm"],
        "round": df["round"].astype(str),
        "best_of_5": df["best_of_5"],
        "surface": df["surface_norm"],
        "indoor": df["indoor_flag"],
        "player_a_id": player_a_id,
        "player_b_id": player_b_id,
        "winner_id": df["winner_id"].astype(str),
        "duration_minutes": df["minutes"],
        "player_a_country": player_a_country,
        "player_b_country": player_b_country,
    })
    matches = matches.dropna(subset=["date"])
    matches = matches[matches["surface"].isin(["hard","clay","grass","indoor-hard"])]

    # Break points convertits (del rival: faced - saved)
    w_bp_conv = (pd.to_numeric(df["l_bpFaced"], errors="coerce") - pd.to_numeric(df["l_bpSaved"], errors="coerce")).clip(lower=0)
    l_bp_conv = (pd.to_numeric(df["w_bpFaced"], errors="coerce") - pd.to_numeric(df["w_bpSaved"], errors="coerce")).clip(lower=0)

    pts_w = pd.DataFrame({
        "match_id": df["match_id"],
        "player_id": df["winner_id"].astype(str),
        "aces": pd.to_numeric(df["w_ace"], errors="coerce"),
        "double_faults": pd.to_numeric(df["w_df"], errors="coerce"),     # <-- arreglat
        "first_sv_in": pd.to_numeric(df["w_1stIn"], errors="coerce"),
        "first_sv_pts_won": pd.to_numeric(df["w_1stWon"], errors="coerce"),
        "second_sv_pts_won": pd.to_numeric(df["w_2ndWon"], errors="coerce"),
        "bp_faced": pd.to_numeric(df["w_bpFaced"], errors="coerce"),
        "bp_saved": pd.to_numeric(df["w_bpSaved"], errors="coerce"),
        "bp_opp": pd.to_numeric(df["l_bpFaced"], errors="coerce"),
        "bp_conv": w_bp_conv,
        "service_games": pd.to_numeric(df["w_SvGms"], errors="coerce"),
        "return_games": pd.to_numeric(df["l_SvGms"], errors="coerce"),
        "hold_games_won": (pd.to_numeric(df["w_SvGms"], errors="coerce") - l_bp_conv).clip(lower=0),
        "break_games_won": w_bp_conv,
        "tb_played": np.nan,
        "tb_won": np.nan
    })

    pts_l = pd.DataFrame({
        "match_id": df["match_id"],
        "player_id": df["loser_id"].astype(str),
        "aces": pd.to_numeric(df["l_ace"], errors="coerce"),
        "double_faults": pd.to_numeric(df["l_df"], errors="coerce"),
        "first_sv_in": pd.to_numeric(df["l_1stIn"], errors="coerce"),
        "first_sv_pts_won": pd.to_numeric(df["l_1stWon"], errors="coerce"),
        "second_sv_pts_won": pd.to_numeric(df["l_2ndWon"], errors="coerce"),
        "bp_faced": pd.to_numeric(df["l_bpFaced"], errors="coerce"),
        "bp_saved": pd.to_numeric(df["l_bpSaved"], errors="coerce"),
        "bp_opp": pd.to_numeric(df["w_bpFaced"], errors="coerce"),
        "bp_conv": l_bp_conv,
        "service_games": pd.to_numeric(df["l_SvGms"], errors="coerce"),
        "return_games": pd.to_numeric(df["w_SvGms"], errors="coerce"),
        "hold_games_won": (pd.to_numeric(df["l_SvGms"], errors="coerce") - w_bp_conv).clip(lower=0),
        "break_games_won": l_bp_conv,
        "tb_played": np.nan,
        "tb_won": np.nan
    })

    points = pd.concat([pts_w, pts_l], ignore_index=True)
    return matches, points


def fetch_tml(year_start:int, year_end:int, include_ongoing:bool=True):
    frames = []
    for y in range(year_start, year_end+1):
        url = f"{RAW_BASE}/{y}.csv"
        try:
            df = fetch_csv(url)
            df["__year"] = y
            frames.append(df)
            print(f"[OK] {y}: {len(df)} rows")
        except Exception as e:
            print(f"[WARN] {y}: {e}")
    if include_ongoing:
        url = f"{RAW_BASE}/ongoing_tourneys.csv"
        try:
            df_ongo = fetch_csv(url)
            df_ongo["__year"] = int(dt.date.today().year)
            frames.append(df_ongo)
            print(f"[OK] ongoing_tourneys: {len(df_ongo)} rows")
        except Exception as e:
            print(f"[WARN] ongoing_tourneys: {e}")
    if not frames:
        raise RuntimeError("No TML data fetched")
    return pd.concat(frames, ignore_index=True)


## 3) Enriquiment opcional (indoor overrides, país per ciutat)

In [13]:

# Llista curada d'esdeveniments ATP habitualment 'indoor hard'
INDOOR_HARD_TOURNAMENTS = {
    # Masters & finals
    "Paris": True, "Turin": True, "London": True, "Milan": True,
    # ATP 500/250 europeus tardor-hivern
    "Basel": True, "Vienna": True, "Rotterdam": True, "Marseille": True, "Montpellier": True,
    "Metz": True, "Sofia": True, "Antwerp": True, "Stockholm": True, "Lyon": True,
    # Històrics recents
    "St. Petersburg": True, "Moscow": True, "Cologne": True, "Astana": True, "Nur-Sultan": True
}

CITY_TO_COUNTRY = {
    "Melbourne":"AUS","Sydney":"AUS","Adelaide":"AUS","Brisbane":"AUS",
    "Paris":"FRA","Lyon":"FRA","Marseille":"FRA","Metz":"FRA","Montpellier":"FRA",
    "London":"GBR","Birmingham":"GBR","Eastbourne":"GBR","Nottingham":"GBR","Queen's Club":"GBR",
    "Madrid":"ESP","Barcelona":"ESP","Mallorca":"ESP","Valencia":"ESP","Sevilla":"ESP",
    "Rome":"ITA","Milan":"ITA","Turin":"ITA","Florence":"ITA","Naples":"ITA","Palermo":"ITA",
    "Vienna":"AUT","Kitzbuhel":"AUT","Kitzbühel":"AUT",
    "Basel":"CHE","Geneva":"CHE","Gstaad":"CHE",
    "Rotterdam":"NLD","s-Hertogenbosch":"NLD","Amsterdam":"NLD","The Hague":"NLD",
    "Antwerp":"BEL","Brussels":"BEL",
    "Stockholm":"SWE","Bastad":"SWE","Båstad":"SWE","Gothenburg":"SWE",
    "Hamburg":"DEU","Halle":"DEU","Stuttgart":"DEU","Munich":"DEU","Cologne":"DEU","Berlin":"DEU",
    "Doha":"QAT","Dubai":"UAE","Abu Dhabi":"UAE",
    "Indian Wells":"USA","Miami":"USA","Cincinnati":"USA","Washington":"USA","Atlanta":"USA","Winston-Salem":"USA","Newport":"USA","Delray Beach":"USA","Dallas":"USA","Houston":"USA","San Jose":"USA",
    "Toronto":"CAN","Montreal":"CAN","Vancouver":"CAN","Quebec City":"CAN",
    "Acapulco":"MEX","Los Cabos":"MEX","Guadalajara":"MEX",
    "Buenos Aires":"ARG","Cordoba":"ARG","Santiago":"CHL","Rio de Janeiro":"BRA","Sao Paulo":"BRA","Quito":"ECU",
    "Estoril":"PRT","Cascais":"PRT","Oeiras":"PRT",
    "Tokyo":"JPN","Osaka":"JPN",
    "Beijing":"CHN","Shanghai":"CHN","Zhuhai":"CHN","Chengdu":"CHN","Shenzhen":"CHN","Wuhan":"CHN","Nanchang":"CHN","Tianjin":"CHN",
    "Seoul":"KOR","Busan":"KOR",
    "Bangkok":"THA","Singapore":"SGP","Kuala Lumpur":"MYS","Pune":"IND",
    "Casablanca":"MAR","Marrakech":"MAR","Tunis":"TUN","Doha":"QAT",
    "Baku":"AZE","Astana":"KAZ","Nur-Sultan":"KAZ",
    "Basel":"CHE","Geneva":"CHE",
}
def enrich_matches(matches: pd.DataFrame) -> pd.DataFrame:
    m = matches.copy()
    # Country by city (best-effort)
    m["country"] = m.apply(lambda r: CITY_TO_COUNTRY.get(str(r["city"]), r.get("country","")), axis=1)
    # Indoor overrides for hard-surface city names
    override = m["city"].map(lambda x: INDOOR_HARD_TOURNAMENTS.get(str(x), False))
    m.loc[(m["surface"]=="hard") & (override.fillna(False)), ["surface","indoor"]] = ["indoor-hard", 1]
    return m


## 4) UI — Paràmetres i Execució

In [14]:

try:
    year_start = widgets.IntText(value=2018, description="Any inicial:")
    year_end   = widgets.IntText(value=2025, description="Any final:")
    include_ongoing = widgets.Checkbox(value=True, description="Incloure ongoing")
    random_flip = widgets.Checkbox(value=True, description="Random flip A/B")
    seed = widgets.IntText(value=2025, description="Seed")
    data_dir = widgets.Text(value="data", description="Data dir:")
    do_enrich = widgets.Checkbox(value=True, description="Enriquir (indoor/country)")
    run_btn = widgets.Button(description="Fetch & Build", button_style="primary")
    out = widgets.Output()

    def on_click(_):
        with out:
            out.clear_output()
            print("Baixant TML...")
            df_raw = fetch_tml(year_start.value, year_end.value, include_ongoing.value)
            print("Construint taules...")
            matches, points = build_from_tml(df_raw, seed=seed.value, do_flip=random_flip.value)
            if do_enrich.value:
                print("Enriquint matches...")
                matches = enrich_matches(matches)
            os.makedirs(data_dir.value, exist_ok=True)
            mpath = os.path.join(data_dir.value, "matches.csv")
            ppath = os.path.join(data_dir.value, "points_sets_games.csv")
            # upsert
            if os.path.exists(mpath):
                old = pd.read_csv(mpath, dtype=str)
                mm = pd.concat([old, matches.astype(str)]).drop_duplicates(subset=["match_id"], keep="last")
            else:
                mm = matches.astype(str)
            mm.to_csv(mpath, index=False)

            if os.path.exists(ppath):
                oldp = pd.read_csv(ppath, dtype=str)
                pp = pd.concat([oldp, points.astype(str)]).drop_duplicates(subset=["match_id","player_id"], keep="last")
            else:
                pp = points.astype(str)
            pp.to_csv(ppath, index=False)

            print(f"✅ Guardat: {mpath}  ({len(mm)} files)")
            print(f"✅ Guardat: {ppath}  ({len(pp)} files)")
            display(mm.head(5))
            display(pp.head(5))

    run_btn.on_click(on_click)

    widgets.VBox([
        widgets.HBox([year_start, year_end]),
        widgets.HBox([include_ongoing, random_flip, do_enrich]),
        widgets.HBox([seed, data_dir]),
        run_btn,
        out
    ])
except Exception as e:
    print("UI no disponible, executa manualment les funcions fetch_tml(...) i build_from_tml(...)")


In [16]:
# Paràmetres
YEAR_START = 2018
YEAR_END   = 2025
INCLUDE_ONGOING = True
RANDOM_FLIP = True
SEED = 2025
DATA_DIR = "."

# 1) baixa i concatena TML
df_raw = fetch_tml(YEAR_START, YEAR_END, include_ongoing=INCLUDE_ONGOING)

# 2) construeix taules
matches, points = build_from_tml(df_raw, seed=SEED, do_flip=RANDOM_FLIP)

# 3) enriquiment (opc.)
matches = enrich_matches(matches)

# 4) desa (amb upsert)
import os, pandas as pd
os.makedirs(DATA_DIR, exist_ok=True)
mpath = os.path.join(DATA_DIR, "matches.csv")
ppath = os.path.join(DATA_DIR, "points_sets_games.csv")

if os.path.exists(mpath):
    old = pd.read_csv(mpath, dtype=str)
    mm = pd.concat([old, matches.astype(str)]).drop_duplicates(subset=["match_id"], keep="last")
else:
    mm = matches.astype(str)
mm.to_csv(mpath, index=False)

if os.path.exists(ppath):
    oldp = pd.read_csv(ppath, dtype=str)
    pp = pd.concat([oldp, points.astype(str)]).drop_duplicates(subset=["match_id","player_id"], keep="last")
else:
    pp = points.astype(str)
pp.to_csv(ppath, index=False)

print(f"✅ Guardat: {mpath} (files: {len(mm)})")
print(f"✅ Guardat: {ppath} (files: {len(pp)})")
display(mm.head(5))
display(pp.head(5))


[OK] 2018: 2926 rows
[OK] 2019: 2806 rows
[OK] 2020: 1466 rows
[OK] 2021: 2735 rows
[OK] 2022: 2918 rows
[OK] 2023: 2995 rows
[OK] 2024: 3076 rows
[OK] 2025: 2648 rows
[OK] ongoing_tourneys: 81 rows
✅ Guardat: ./matches.csv (files: 21598)
✅ Guardat: ./points_sets_games.csv (files: 43302)


,match_id,date,tournament,city,country,level,round,best_of_5,surface,indoor,player_a_id,player_b_id,winner_id,duration_minutes,player_a_country,player_b_country
0,2018-339_1.0_2018-01-01,2018-01-01,Brisbane,Brisbane,AUS,ATP,R32,0,hard,0,MH30,PB22,MH30,95.0,AUS,CAN
1,2018-339_2.0_2018-01-01,2018-01-01,Brisbane,Brisbane,AUS,ATP,R32,0,hard,0,SU55,E831,E831,150.0,CAN,GBR
2,2018-339_3.0_2018-01-01,2018-01-01,Brisbane,Brisbane,AUS,ATP,R32,0,hard,0,MA30,CH27,CH27,99.0,LUX,KOR
3,2018-339_4.0_2018-01-01,2018-01-01,Brisbane,Brisbane,AUS,ATP,R32,0,hard,0,E690,TD51,E690,59.0,AUS,USA
4,2018-339_5.0_2018-01-01,2018-01-01,Brisbane,Brisbane,AUS,ATP,R32,0,hard,0,E873,Z184,Z184,72.0,MEX,ARG


,match_id,player_id,aces,double_faults,first_sv_in,first_sv_pts_won,second_sv_pts_won,bp_faced,bp_saved,bp_opp,bp_conv,service_games,return_games,hold_games_won,break_games_won,tb_played,tb_won
0,2018-339_1.0_2018-01-01,MH30,8.0,0.0,37.0,27.0,17.0,1.0,1.0,10.0,3.0,9.0,9.0,9.0,3.0,nan,nan
1,2018-339_2.0_2018-01-01,E831,5.0,5.0,65.0,54.0,29.0,6.0,5.0,3.0,2.0,17.0,17.0,16.0,2.0,nan,nan
2,2018-339_3.0_2018-01-01,CH27,7.0,4.0,35.0,29.0,17.0,2.0,2.0,5.0,2.0,10.0,11.0,10.0,2.0,nan,nan
3,2018-339_4.0_2018-01-01,E690,7.0,1.0,32.0,23.0,11.0,3.0,2.0,5.0,4.0,9.0,8.0,8.0,4.0,nan,nan
4,2018-339_5.0_2018-01-01,Z184,12.0,3.0,37.0,31.0,11.0,1.0,1.0,6.0,2.0,10.0,9.0,10.0,2.0,nan,nan
